<a href="https://colab.research.google.com/github/ashalem/ML_Human/blob/main/WS3_2024_students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>Machine Learning and Human Behavior - 236667 - Winter 2024-2025</div>
<font size="6">Workshop #3 - Biased inputs, biased outputs 🦡</font>

# Instructions and submission guidelines

* Clone this notebook and complete the exercise:
    * Aim for clear and concise solutions.
    * Indicate clearly with a text block the sections of your solutions.
    * Answer dry questions in text (markdown) blocks, and wet questions in code blocks.
* Submission guidelines:
    * When you're done, restart the notebook, and make sure that everything runs smoothly (Runtime->"Restart and Run All")
    * Add a text block in the beginning of your notebook with your IDs.
    * Export your notebook as ipynb (File->Download->"Download .ipynb")
    * If you need to attach additional files to your submission (e.g images), add them to a zip file together with the notebook ipynb file.
    * Submit through the course website. Remember to list partner IDs when you submit.
* **Due date**: Sunday 5/1/2025, 10:00
* For any questions regarding this workshop task, contact [Lotan](mailto:lotan.amit@campus.technion.ac.il).


## Preliminaries
Run these cells to load into memory interface objects and functions that will be used throughtout today's workshop.

No need to read the actual code.

In [ ]:
from typing import Iterable, Set, Tuple
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import ndcg_score, precision_score, recall_score, mean_squared_error
from scipy.stats import bernoulli, binom, norm
import uuid
from typing import List, Tuple
import seaborn as sns


from tqdm.auto import tqdm

CLICKS_NOISE_SCALE = 5
RATINGS_NOISE_SCALE = 2
LATENT_DIMENSION = 30

In [ ]:
#@title RateClickEnvironment

class RateClickEnvironment(object):
  FEATURE_COLUMNS = [f'x{i+1}' for i in range(8)]

  def __init__(self, thresh_click=2, eps_click=0.4,
               eps_rating=1, user_mu=0, user_sigma=1):
    self.thresh_click = thresh_click
    self.eps_click = eps_click
    self.eps_rating = eps_rating
    self._latent_dim = len(self.FEATURE_COLUMNS)
    self.top_k_idxs = np.arange(self._latent_dim)<5

    self._user_features = np.abs(np.random.normal(user_mu, user_sigma, self._latent_dim))# np.abs(np.random.randn(self._latent_dim))
    self._user_features /= np.linalg.norm(self._user_features)
    self._user_features_topk = self._user_features*self.top_k_idxs

    self.ndcg_10 = lambda y_true, y_est: ndcg_score(
    np.expand_dims(y_true, 0),
    np.expand_dims(y_est, 0),
    k=10,
    )

  def _generate_dataframe(self, n_samples, seed=None) -> pd.DataFrame:
    if seed is not None:
      np.random.seed(seed)
      print('in')
    return pd.DataFrame(
        index=pd.Index(
            data=[str(uuid.uuid4()) for _ in range(n_samples)],
            name='item_id',
        ),
        data={
            feature_name: np.random.randn(n_samples)
            for feature_name in self.FEATURE_COLUMNS
        }
    )

  @staticmethod
  def _click_transform(x):
    return x>2

  @staticmethod
  def _rating_transform(x):
    sigmoid = lambda x: 1/(1+np.exp(-x))
    return np.round((sigmoid(x*1.5)-0.5)*4 + 3).astype(int)

  def generate_evaluation_set(self, n_samples: int, seed: int=None) -> pd.DataFrame:
    df = self._generate_dataframe(n_samples, seed)
    X = df[self.FEATURE_COLUMNS].to_numpy()
    scores = X@self._user_features
    y = self._rating_transform(scores)
    return df.assign(true_relevance=y)

  def generate_training_set(self, n_samples: int, seed: int=None):
    df = self._generate_dataframe(n_samples, seed)
    X = df[self.FEATURE_COLUMNS].to_numpy()
    for _ in range(1000):  # rejection sampling
      noisy_scores = X@self._user_features_topk + self.eps_click*np.random.logistic(size=X.shape[0])
      y_click = pd.array(
          (noisy_scores > self.thresh_click),
          dtype=pd.Int64Dtype()
      )
      if len(y_click.unique())>1 and y_click.value_counts().min()>1:
        break
    for _ in range(1000):  # rejection sampling
      noisy_scores = X@self._user_features + self.eps_rating*np.random.logistic(size=X.shape[0])
      y_rating = pd.array(
          self._rating_transform(noisy_scores),
          dtype=pd.Int64Dtype(),
      )
      y_rating[y_click==0] = None
      if len(y_rating.dropna().unique())>1:
        break

    return df.assign(click=y_click, rating=y_rating)

  def build_X_y_matrices(
    self,
    df: pd.DataFrame,
    label_column: str,
    feature_columns: List[str]=FEATURE_COLUMNS,
    get_valid_entries:bool=False ) -> Tuple[np.ndarray, np.array]:
    valid_entries = pd.notna(df[label_column])
    X = df.loc[valid_entries][feature_columns].to_numpy()
    y = df.loc[valid_entries][label_column].to_numpy()
    if get_valid_entries:
      return X,y,valid_entries
    else:
      return X,y

def generate_population(utype_1='ranting', user_mu_1=-1, user_sigma_1=0.5,
                        thresh_click_1=1, ratio=4, utype_2='extreme',
                        user_mu_2=2, user_sigma_2=2, thresh_click_2=2
                        ):
# utype_1='ranting'
# user_mu_1=-1
# user_sigma_1=0.5
# thresh_click_1=1
# ratio=4
# utype_2='extreme'
# user_mu_2=2
# user_sigma_2=2
# thresh_click_2=2.5


  train_size_ma = 10000 * 3
  train_size_min = 10000

  rc_env_rant = RateClickEnvironment(user_mu=user_mu_1, user_sigma=user_sigma_1,
                                    thresh_click=thresh_click_1)
  train_df = rc_env_rant.generate_training_set(train_size_ma)
  items_train_rant, ratings_train_rant = rc_env_rant.build_X_y_matrices(train_df, 'rating')
  test_df = rc_env_rant.generate_evaluation_set(500)
  items_test_rant, true_relevance_rant = rc_env_rant.build_X_y_matrices(test_df, 'true_relevance')
  ratings_train_rant, items_train_rant = selective_ratings(ratings_train_rant,
                                                          utype_1,items_train_rant)

  # extreme reporters
  rc_env_over = RateClickEnvironment(user_mu=user_mu_2, user_sigma=user_sigma_2,
                                    thresh_click=thresh_click_2)
  train_df = rc_env_over.generate_training_set(train_size_min)
  items_train, ratings_train = rc_env_over.build_X_y_matrices(train_df, 'rating')
  test_df = rc_env_over.generate_evaluation_set(500)

  items_test_over, true_relevance_over = rc_env_over.build_X_y_matrices(test_df, 'true_relevance')
  ratings_train_over, items_train_over = selective_ratings(ratings_train,
                                                          utype_2,items_train)

  # merge:
  rant_df = pd.DataFrame(data=np.hstack([items_train_rant,
                                      np.expand_dims(ratings_train_rant,1)]),
                      columns=[f'x_{i}' for i in range(items_train_rant.shape[1])] + ['rating'])

  over_df = pd.DataFrame(data=np.hstack([items_train_over,
                                        np.expand_dims(ratings_train_over,1)]),
                        columns=[f'x_{i}' for i in range(items_train_over.shape[1])] + ['rating'])

  train_df = pd.concat([rant_df, over_df])
  train_df['user_type'] = np.hstack([np.ones(ratings_train_rant.shape),
                                    np.zeros(ratings_train_over.shape)])

  rant_df = pd.DataFrame(data=np.hstack([items_test_rant,
                                        np.expand_dims(true_relevance_rant,1)]),
                        columns=[f'x_{i}' for i in range(items_train_rant.shape[1])] + ['true_relevance'])

  over_df = pd.DataFrame(data=np.hstack([items_test_over,
                                        np.expand_dims(true_relevance_over,1)]),
                        columns=[f'x_{i}' for i in range(items_test_over.shape[1])] + ['true_relevance'])

  test_df = pd.concat([rant_df, over_df])
  test_df['user_type'] = np.hstack([np.ones(true_relevance_rant.shape), np.zeros(true_relevance_over.shape)])

  return train_df, test_df


# Task 1: Predictions through the funnel

## Introduction

In this exercise, we will simulate and analyze a hypothetical recommender system. For simplicty, our system will have a single user, and the goal of the system is to provide her with relevant content.
The user is described by a (latent) vector $u\in \mathbb{R}^d$, which is *unknown* to the system. For a given item $i$ with features $x_i \in \mathbb{R}^d$ (which are *known* to the system), relevance to the user is an integer $y \in \{1,\dots,5\}$, given by the following formula:
$$
y=\texttt{round}(\sigma(u^\top x_i)-0.5) \quad \text{where} \quad
\sigma(a)=1+\frac{4}{1+e^{-1.5a}}
$$
Intuitively, the formula computes an inner product, "squashes" the result into $[1,5]$ using a scaled sigmoid function, and rounds.

To make its recommendations, **the system is given access to two types of data**, generated by the user as a response to past recommendations: clicks, and (possibly) ratings.

Given a training set of previously-recommended items, for each of these items, the system observes:
  1. A binary *click* label $c \in \{0,1\}$. The user clicks on a recommended item only if its *perceived* relevance $\tilde{y}$ is higher than some threshold `thresh_click`. Perceived relevance differs from true relevance in two ways:
    1. It is based only on partial information (e.g think of how search results include only a short summary of an item). In our case, the user only gets to observe some of the features.
    2. It includes additive noise `eps_click` (this is applied to the inner product $u^\top x_i$).
  2. An integer *rating* label $r \in \{1,\dots,5\}$. Ratings are based on items relevancies, and are reported *only after items have been consumed*. This means that:
    1. They are based on the entire feature vector (i.e., consuming an items means the user is exposed to the entire feature vector).
    2. They are also affected by additive noise `eps_rating`.
    3. Importantly, they are observed **only for items that have been clicked**.

None of the parameters `thresh_click`, `eps_click`, and `eps_rating` are known to the system.

In this exercise, we will explore how the interaction between these two types of user feedback affect learning and recommendation.


###  `RateClickEnvironment` Interface


`RateClickEnvironment` is an interface which generates synthetic datasets of clicks and ratings according to the paradigm above. The constructor includes three arguments: `thresh_click`, `eps_click`, and `eps_rating`, as described above. Calling `RateClickEnvironment()` results in default arguments.

Additionally, it has two public instance methods that we're going to use:

* `generate_evaluation_set(n_samples)` - Generate evaluation set of the given size. This function returns a `pandas.DataFrame` object. The dataframe is indexed by `item_id`, which is an arbitrary random string assigned to each item. The dataframe has columns for each of the $d$ features, and additional `true_relevance` column that stores the true (un-noised) relevancies of all items.

* `generate_training_set(n_samples)` - Generate a training set of the given size. Similarly, this function returns a `pandas.DataFrame` object with the same type of index, and with feature columns. The dataframe does not include true relevancies; instead it includes two additional columns:
 * The column `click` is an indicator of whether the user clicked an item ($c_{i}\in\{0,1\}$ as in the definitions above).
 * The column `rating` contains the item rating ($r_{i}\in\{1,\dots,5\}$ in the definitions above), or `N/A` if the rating is not observed.


## 1.1 Exploratory analysis: Warmup

`rc_env` is an instance of the `RateClickEnvironment` class:


In [ ]:
rc_env = RateClickEnvironment()

Use the `rc_env.generate_training_set` method to generate a training set of size 1000 and print the first 10 rows. Note that values of `rating` are only observed for items that were clicked.

In [ ]:
## YOUR SOLUTION

Calculate the proportion of items that were clicked in the training set (result should be a number between 0 and 1):

(try to do this in one line of pandas code!)

In [ ]:
## YOUR SOLUTION

Plot a bar chart showing the number of times each rating value (1,2,...,5) was observed (histogram). Use `df.value_counts()`.

(try to do this in one line of pandas code!)

In [ ]:
## YOUR SOLUTION

Use the `rc_env.generate_evaluation_set` method with input 1000 to generate an evaluation set with 1000 samples, print the first 5 rows.


In [ ]:
## YOUR SOLUTION

Plot an histogram of true (un-noised) values. Use `df.value_counts()`.


In [ ]:
## YOUR SOLUTION

How is this histogram different than the one plotted for the training set ratings? Why is that? Explain your answer.



```
Answer:...
```



## 1.2 Basic model training

The utility method `rc_env.build_X_y_matrices`
extracts training matrices and labels from a given dataset DataFrame.
Entries for which the labels are missing (`N/A`) are removed from the dataset.

Arguments:
- `label_column (str)`: Name of column to be used as label (e.g 'click')
- `feature_columns (List[str])`: Columns to be used as features



Use `rc_env.build_X_y_matrices` together with previous methods to:
* Create a train set of size 1000 that includes ratings as labels.
* Train a linear regression model (use scikit-learn).
* Create an evaluation set of size 500 (that includes true relevancies $y$).
* Use the learned model to generate predicted relevancies $\hat{y}$
* Create a violin plot comparing $\hat{y}$ to $\hat{y}$ (code provided).

**Note:** You may try to re-run with different seeds.





In [ ]:

## YOUR SOLUTION

In [ ]:
# [plot correlations]

# eval_r: true ratings(1d numpy.ndarray)
# pred_r: predicted ratings(1d numpy.ndarray)

plt.figure(figsize=(8,6));
rs = np.sort(np.unique(eval_r))
pred_per_r = [pred_r[np.where(eval_r==r)] for r in rs]
plt.violinplot(pred_per_r, rs, showmeans=True);
plt.plot([1,5],[1,5],'k--');
plt.text(5.05, 5.05, 'Perfect \n Pred');
plt.xlabel("True");
plt.ylabel("Predicted");

**Explain the plot -** Does the linear regression model capture the true relevance?



```
Answer:...
```



## 1.3 The effect of dataset size

* Create a `RateClickEnvironment` object with default arguments, and use it to generate an evaluation set of size 500.
* For each n in `geomspace(50,20000,8)`, generate a training dataset of size n.
* Train a click-prediction model using logistic regression.
* Train a rating-prediction model using linear regression.
* Evaluate and store the NDCG of each model on the evaluation set: use the following `rc_nev.ndcg_10` method that wraps scikit-learn's `ndcg_score` for top 10:

  * Input: 2 arrays, true relevance and predicted relevance score.
  * Output: float, score

* For each value of n, **repeat the above procedure 20 times**.
* Generate a results DataFrame with the following columns: `n_samples, repetition, click_ndcg, rating_ndcg`

Generate evaluation set (to be shared across simulation repetitions):

In [ ]:
np.random.seed(42)

## YOUR SOLUTION


Run simulations and output results DataFrame:

In [ ]:
np.random.seed(42)
num_repeats = 20

n_values_lst = np.geomspace(50,20000,8).astype(int)

## YOUR SOLUTION


Plot a line-plot showing the average NDCG per n for each of the two models. Include error bars.

Guidance:

*   Group by trainset size
*   Calculate aggregated mean and std values for columns `click_ndcg` and `rating_ndcg`.
*   Use mean values for the series plots (the lines) and std for the error bars.

Technical tip:
*   After aggregation, your pandas DataFrame may contain multi-level columns, the methods `df.swaplevel(axis=1)` or `df.xs(...,axis=1,level=2)` may be convenient for reshaping the DataFrame before plotting.
* The pandas `plot.line` function has an optinal argument `yerr` for specfying error bar size.



In [ ]:
## YOUR SOLUTION


**Explain the plot -** What can be observed from the graph? In What is the relation between the two lines?

```
Answer: ...
```

## 1.4 Varying click threshold
Repeat the experiment, but this time:
* Keep the number of training samples fixed at 1000.
* Vary the thershold $\mathtt{thresh\_click} \in[0,5]$. Pass this as a parameter to the `RateClickEnvironment` constructor (note this means that now the environment object should be constructed *inside* the loop).
* For every threshold parameter run 20 random repeats and average the results.

(Feel free to reuse code from the above question)

In [ ]:
click_thresholds = np.linspace(0, 5, 20)

## YOUR SOLUTION

Plot Results - a line graph including clicks and ratings ndcg, similar to 1.3 but with threshold vs.ndcg. Again, feel free to reuse code.

In [ ]:
## YOUR SOLUTION


In [ ]:
results_df

**Explain the plot -** What can be observed from the graph? In What is the relation between the two lines?

```
Answer: ...
```

# Task 2: Selective rating

In real life, users rarely rate all items they consume. Moreover - both the choice of *which* items to rate, and the actual *reported* ratings, are prone to behavioral biases. In this question we will examine the effects of different reporting schemes (i.e., ways in which uesrs report or fail to report ratings, and how they modify these ratings) on learning.


We will examine five types of users:
- Rational (random): Each rating $r$ is reported with a fixed probability $p=0.4$. Note this serves as a benchmark since it does not add bias to how ratings are reported.
- Ranting: Only reports bad ratings, defined as $r \le 2$.
- Extreme: Only reports extreme ratings, defined as $r=1$ or $r=5$.
- Optimistic: Reports only if ratings are high, $r \ge 4$, but for each item with true relevance $r=4$, misreports it as $r=5$ with probability $p=0.8$.
- Overreacts: Reports ratings for items with low ratings $r \le 2$ and with high ratings $r \ge 4$, and for these, reports them with probability $p=0.5$. However, he extremifies his ratings by misreporting $r=2$ as $r=1$ and $r=4$ as $r=5$.

Note: the above criteria and probabilities ensure that for all types, ~40% of ratings are reported, so all types generate the same (expected) number of ratings.




### 2.1 Implementing user types
Implement a function `selective_ratings` that takes as input an array `r_arr` of ratings, and simulates user behavior by returning a subset of `r_arr` matching one of the above types of user reporting behaviors, as determined by the parameter `utype`. The function also takes as input the set of item features `items_train`, and returns the subset of items corresponding to the reported ratings.

You may use `rc_env._user_features` to access actual user values per features.

In [ ]:
def selective_ratings(r_arr, utype='rational',items_train=None):
  """
  input:
  r_arr: ratings array - numpy
  utype: string

  return:
    - revealed_ratings: numpy array of revealed ratings
    - revealed_items: features of the corresponding items
  """

  if utype=='rational':

    ## YOUR SOLUTION



  if utype=='ranting':
    ## YOUR SOLUTION


  if utype=='extreme':
    ## YOUR SOLUTION


  if utype=='optimistic':
    ## YOUR SOLUTION


  if utype=='overreacts':
    ## YOUR SOLUTION





#### 2.1.1 New user types
Implement 2 data revelation models of your own, and explain the behavioral rationale behind them. Make sure that ~40% of the data is revealed. You may edit the function above.



Use next block of code to experiment with your ratings filter models

In [ ]:
"""

Try your code -
You may use ratings generated below of the evaluation..

"""

What is the rationale behind the two models? How did you make sure that ~40% of ratings are reported?
```
Answer: ...
```

### 2.2 Training and evaluation

In this section we will:
* generates train and test data
* filter rating for each user type based on your implementation of `selective_ratings`
* Train and evaluate the performance.



2.2.1 **Before actually running the code** - give an educated guess for NDCG and RMSE values for each user type (if values are hard for you to guess, state relations, e.g. 'higher', 'lower', 'much lower'):



```
Answer:...
```



2.2.2 Generate traning set of size 5000 and evaluation set of size 1000. Use `build_X_y_matrices` to get `rating` as label for training and `true_relevance` for the evaluation.

In [ ]:
np.random.seed(42)

## YOUR SOLUTION

For each user type train and evaluate the results. Print ndcg and rmse.

In [ ]:

utypes = ['rational', 'ranting', 'extreme', 'optimistic', 'overreacts'] #todo: add your own

ndcgs = []
rmses = []
preds = []

for utype in utypes:

  ## YOUR SOLUTION

## 2.3 Reflection
#### Comparing guesses and results
Do these results match your guesses? Why, or why not?


To answer fully run the following cell first.
It will generate a violin plots of per user type.

Use these distributions to explain ndcg and rmse preformance.

In [ ]:

for utype_, pred_, ndcg_, rmse_ in zip(utypes, preds, ndcgs, rmses):
  rs = np.sort(np.unique(true_relevance))
  pred_per_r = [pred_[np.where(true_relevance==r)] for r in rs]
  plt.violinplot(pred_per_r, rs, showmeans=True)
  plt.plot([1,5],[1,5],'k--');
  plt.xlabel('true relevance')
  plt.ylabel('predicted rating')
  plt.title('{}: ndcg={:.3f}, rmse={:.3f}'.format(utype_, ndcg_, rmse_))
  plt.text(3.2, 3.8, 'Perfect \n Pred');
  plt.show()

Explain the results:



```
Answer:...
```



## 2.4 Sub Populations and IPW

In this segment, our dataset encompasses two distinct user groups: *Ranters*, which like to complain and *Extreme Rankers* which report extreme enjoyment or disappointment. These groups vary not only in their rating policies but also in user features. Your objective with this imbalanced training set is to learn "true_relevance" based on rated items.

To improve the baseline performance, you'll explore using inverse propensity weighting (IPW). In practical scenarios, you might not know the user type during testing. Propensity weighting can help account for this uncertainty and capture the information effectively.


The function `generate_population` generates train and test dataframes which include items' feature columns and either received `rating` or `true_relevance`. In addition, each dataset has a `user_type` column. A binary column which indicates if the item displayed was rated by a *Ranter* or not.
You may **not** use this column directly in your training process.






In [ ]:
train_df, test_df = generate_population()

## YOUR SOLUTION

As discussed in the lecture, leverage `LogisticRegression` to calculate propensity weights for each item. Using `user_type` is allowed for weights calculation.

Note that`LogisticRegression`default settings includes regularization.

In [ ]:
## YOUR SOLUTION

Use `RandomForestRegressor` model to fit to ratings and predict true relevance.
Calculate baseline preformance and weighted preformance.

Save overall test NDCG, and for each the sub population (e.g NDCG for *Ranters*).


In [ ]:
## YOUR SOLUTION

Use a bar graph to plot results in trems of NDCG. Use to series, to represent weighted and un-weighted preformace for each population (Ranters, Extreme reporters and overall).

Note: changes in preformance don't have to be major.

In [ ]:
## YOUR SOLUTION

**Explain the results:**



```
Answer:...
```



**Explain:** Why would IPW be a viable strategy in this secenario?

```
Answer:...
```